In [1]:
# import pandas as pd
# import numpy as np

# import matplotlib.pyplot as plt 
# #%matplotlib inline 

import seaborn as sns
sns.set(color_codes=True)

from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_score, precision_recall_fscore_support

from sklearn.pipeline import make_pipeline

In [2]:
# All imports required for analysis are below
from __future__ import print_function
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import ipywidgets
from ipywidgets import widgets as wd
import sklearn
# from sklearn import svm
# from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import metrics as mt
from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA 
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime
import glob
import os
from IPython.display import Image
# Functions for ROC Graphs
#https://stackoverflow.com/questions/25009284/how-to-plot-roc-curve-in-python
#probs is the result of a model.predict_proba(x_test) call

#This function plots an ROC curve
def rocCurvePlot(probs, y_test1):
    # calculate the fpr and tpr for all thresholds of the classification
    preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test1, preds)
    roc_auc = metrics.auc(fpr, tpr)

    # method I: plt
    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
# A function to find the optimal cutoff point from ROC curve

#https://stackoverflow.com/questions/28719067/roc-curve-and-cut-off-point-python
def Find_Optimal_Cutoff(target, predicted):
    """Parameters:
    target : Matrix with dependent or target data, where rows are observations
    predicted : Matrix with predicted data, where rows are observations
    ----------
    Return: list type, with optimal cutoff value
    """
    predicted = predicted[:,1]
    fpr, tpr, threshold = metrics.roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

In [ ]:
#directory = 'C:/Users/N1110/Desktop/QTW/QTW_Final/'
directory = 'C:/Users/danie/Documents/GitHub/Quantifying-The-World/Final'
df_raw = pd.read_csv('final_project.csv')
df_raw.head()

In [ ]:
#data cleaning
df_raw['x32'] = df_raw['x32'].str.replace('%', '')
df_raw['x32'] = df_raw['x32'].astype(float)


In [ ]:
df_raw['x37'] = df_raw['x37'].str.replace('$', '')
df_raw['x37'] = df_raw['x37'].astype(float)

In [ ]:
# x34, x33, x10, x23 have 3% missing values, others have below 2%. 
# we should still have enough data after drop a small percentage of NA

In [ ]:
df_raw.dropna(inplace = True)
df_raw.shape

In [ ]:
# x29 only keep 3 characters
df_raw['x29'].replace(to_replace='January', value='Jan',inplace=True)
df_raw['x29'].replace(to_replace='sept.', value='Sep',inplace=True)
df_raw['x29'].replace(to_replace='Dev', value='Dec',inplace=True)
df_raw['x29'].replace(to_replace='July', value='Jul',inplace=True)
# x32 change to float
if(df_raw['x32'].dtypes!='float64'):
    df_raw['x32'] =df_raw['x32'].str[:-1].astype(float)
# x37 change to float
if(df_raw['x37'].dtypes!='float64'):
    df_raw['x37'] = df_raw['x37'].str.replace('$', '')
    df_raw['x37'] =df_raw['x37'].str[:-1].astype(float)

In [ ]:
#check all obj attributes
for col in list(df_raw.select_dtypes(include=['object']).columns):
    print(col)
    print('------------------------------------------------')
    print(df_raw[col].value_counts().sort_values(),'\n')

In [ ]:
#Isolate remaining categorical variables
begColumnCt = len(df_raw.columns)
sD_nominal = df_raw.loc[:, (df_raw.dtypes == object)]

#one hot encode categorical variables
df_raw = pd.get_dummies(data=df_raw, 
                       columns=sD_nominal.columns, drop_first=True)

#Determine change in column count
endColumnCt = len(df_raw.columns)
columnsAdded = endColumnCt - begColumnCt

## XGB

In [3]:
df_imputed = pd.read_csv("df_imputedUpdated2.csv")  #start with window
df_imputed.head(5)

,Unnamed: 0,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x29_Jun,x29_Mar,x29_May,x29_Nov,x29_Oct,x29_Sep,x30_monday,x30_thurday,x30_tuesday,x30_wednesday
0,39909,-0.200526,-1.498802,12.502326,-0.086056,2.648814,5.918173,18.177443,-27.667335,14.412525,...,1,0,0,0,0,0,0,0,0,1
1,122788,-0.025395,4.405048,16.630177,-15.360624,1.049496,-2.587212,24.179028,9.579218,-5.079642,...,1,0,0,0,0,0,0,0,0,1
2,132992,-0.800132,-2.275084,12.319186,0.368629,-1.998036,3.005167,17.911171,4.069285,-3.997345,...,0,0,0,0,1,0,0,1,0,0
3,73551,0.461256,-2.348442,-25.594506,8.090648,-0.599051,24.120313,-37.212488,16.707442,2.629158,...,0,0,0,0,0,0,0,0,0,1
4,148086,-0.418037,2.361275,11.605771,-3.929155,-6.374405,-1.792881,16.873919,-29.730196,-14.898756,...,0,0,0,0,0,0,0,0,0,1


In [4]:
df_imputed.shape

(31678, 66)

In [5]:
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'y' in df_imputed:
    y = df_imputed['y'].values # get the labels we want
    del df_imputed['y'] # get rid of the class label
    X = df_imputed.values # use everything else to predict!

    ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


In [6]:
## Training and Testing Split
# okay, so run through the cross validation loop and set the training and testing variable for one single iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]

    # we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)
# to Make things easier, let's start by just using whatever was last stored in the variables:
##    X_train , y_train , X_test, y_test (they were set in a for loop above)
from sklearn.preprocessing import StandardScaler
scl_obj = StandardScaler()

scl_obj.fit(X_train)
X_test_scaled = scl_obj.transform(X_test)

X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) 

In [ ]:
#baseline
import xgboost as xgb


# #XGBOOST parameters 1
# MAX_ROUNDS = 1000
# EARLY_STOP = 50
# OPT_ROUNDS = 1000
# VERBOSE_EVAL = 50
# RANDOM_STATE = 2000

# #XGBOOST transform data into DMatrix format for modeling
# dtrain = xgb.DMatrix(X_train_scaled, y_train)
# dvalid = xgb.DMatrix(X_test_scaled, y_test)
# type(dtrain)


# # XGBoost Parameters 2
# params = {}
# params['objective'] = 'binary:logistic'
# #params['objective'] = 'multi:softmax'
# #params['objective'] = 'reg:linear'
# params['eta'] = 0.039
# params['silent'] = True
# params['max_depth'] = 2
# params['subsample'] = 0.8
# params['colsample_bytree'] = 0.9
# params['eval_metric'] = 'auc'
# params['random_state'] = RANDOM_STATE

# watchlist = [(dtrain, 'train'), (dvalid, 'valid')]



In [ ]:
xgb_clf = xgb.train(params,
                   dtrain,
                   MAX_ROUNDS,
                   watchlist,
                   early_stopping_rounds = EARLY_STOP,
                   maximize = True,
                   verbose_eval = VERBOSE_EVAL)

In [ ]:
preds = xgb_clf.predict(dvalid)
print('XGBoost - roc_auc_score: ', roc_auc_score(y_test, preds)) 

# XGBoost - roc_auc_score:  0.9207827685624094
# XGBoost - roc_auc_score:  0.9128279684449344

In [ ]:
#type(preds)
# preds = pd.DataFrame(preds, columns=['y_Predicted'])
# y_test = pd.DataFrame(y_test, columns=['y_Actual'])

In [ ]:
preds

In [ ]:
confusion_matrix = pd.crosstab(y_test['y_Actual'], preds['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

In [ ]:
cm

In [ ]:
#Confusion Matrix for best XGB
cm = pd.crosstab(y_test, preds, rownames = ['Actual'], colnames = ['Predicted'])
fig, (ax1) = plt.subplots(ncols = 1, figsize = (5, 5))
sns.heatmap(cm, xticklabels = ['y_0', 'y_1'], yticklabels = ['y_0', 'y_1'], 
            annot = True, ax = ax1,
            linewidths = .2, linecolor = 'Darkblue', cmap ='Blues')
plt.title('confusion Matrix', fontsize = 14)
plt.show()

In [ ]:
total_accuracy_xgb = mt.accuracy_score(y_test, preds)
total_precision_xgb = precision_score(y_test, preds)
classification_rpt_xgb = classification_report(y_test, preds)
print ("XGB accuracy with corresponding parameter settings")
print('XGB classifier accuracy with optimal parameters is: %.3f'%(total_accuracy_xgb))
print('XGB classifier precision with optimal parameters is: %.3f'%(total_precision_xgb))
print('XGB classifier classsification report with optimal parameters is: \n',(classification_rpt_xgb))

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
# A parameter grid for XGBoost
#at least include the parameters already included in the baseline model for chance to find one better than base model.
params = {
        'min_child_weight': [1, 5],
        'gamma': [0.5, 1, 2],
        'subsample': [0.6, 0.8],
        'colsample_bytree': [0.6, 0.9],
        'max_depth': [2, 5]
        }

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime

folds = 3
param_comb = 3

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

#random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train_scaled,y_train), verbose=50, random_state=2000)

xgb_algo = xgb.XGBClassifier()
random_search = RandomizedSearchCV(xgb_algo,
                                   param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train_scaled,y_train), verbose=50, random_state=2000)

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train_scaled, y_train)
timer(start_time) # timing ends here for "start_time" variable

In [ ]:
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)

In [ ]:
# 0.9013276226602549

In [ ]:
# start_time = timer(None)

xgb_best = XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing= None, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.6, tree_method=None,
              validate_parameters=False, verbosity=None)

xgb_best.fit(X_train_scaled, y_train)

# timer(start_time)

In [ ]:
# timer the testing time
start_time = timer(None)
preds = xgb_best.predict(X_test_scaled)
timer(start_time)
print('XGB - roc_auc_score: ', roc_auc_score(y_test, preds)) 

In [ ]:
# parameter searching did not improve the roc auc because used very limited parameter grid.

In [ ]:
# calculate cost

# cost function:
#  Total Cost = 10 * Count of FP + 500 * Count of FN